In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ekphrasis


Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import csv
from ekphrasis.classes.preprocessor import TextPreProcessor
#from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from ekphrasis.classes.segmenter import Segmenter
from nltk.tokenize import TweetTokenizer

data = []
target = []
id = []
for line in open("/Users/ooduyebo/Desktop/task1_training_no_utf.tsv"):
    line_sep = line.split('\t')

    data.append(line_sep[3])
    target.append(line_sep[2])
    id.append(line_sep[0])


text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'money', 'phone',
        'time', 'date'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens

    # corpus from which the word statistics are going to be used
    # for word segmentation
    segmenter="twitter",

    # corpus from which the word statistics are going to be used
    # for spell correction
    corrector="twitter",

    unpack_hashtags=True,  # perform word segmentation on hashtags
    # unpack_users=True, # dunno if this is a thing
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    #tokenizer=SocialTokenizer(lowercase=True).tokenize,
    tokenizer=TweetTokenizer().tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

seg = Segmenter(corpus="twitter")

clean_tweets = []
for tweet in data:

    # manually tag usernames
    # ex: @DoctorChristian -> <user> doctor christian </user>
    match = re.findall(r'@\w+', tweet)

    try:
       for at in match:
           user_seg = seg.segment(at[1:])
           tweet = tweet.replace(at, '<user> ' + user_seg + ' </user>')
    except:
        None

    # manually tag all caps so that the unpack_contractions functions works
    match = re.findall(r"(?<![#@$])\b([A-Z][A-Z ,.']*[A-Z])\b", tweet)

    try:
        for all_caps in match:
            tweet = tweet.replace(all_caps, '<allcaps> ' + all_caps.lower() + ' </allcaps>')
    except:
        None

    match = re.findall(r"(\d+.?\d?%)", tweet)

    try:
        for percent in match:
            tweet = tweet.replace(percent, '<percent> ' + percent[0:len(percent)-1] + ' </percent>')
    except:
        None

    # deal with contractions that the tool misses
    tweet = re.sub(r"(\b)([Ww]hat|[Ii]t|[Hh]e|[Ss]he|[Tt]hat|[Tt]here|[Hh]ow|[Ww]ho|[Hh]ere|[Ww]here|[Ww]hen)'s", r"\1\2 is", tweet)
    tweet = re.sub(r"(\b)([Aa]in)'t", r"is not", tweet)
    tweet = re.sub(r"(\b)([Ww]asn)'t", r"was not", tweet)
    tweet = re.sub(r"(\b)([Hh]e|[Ss]he|[Ii]|[Yy]ou|[Tt]hey|[Ww]e)'d", r"\1\2 would", tweet)
    tweet = re.sub(r"(\b)([Ii]t|[Tt]hat|[Tt]his)'ll", r"\1\2 will", tweet)
    tweet = re.sub(r"(\b)([Cc])'mon", r"come on", tweet)

    # process the rest of the tweet with the nltk tweet tokenizer
    tweet = " ".join(text_processor.pre_process_doc(tweet)).lower()

    clean_tweets.append(tweet)

# below is code to create the tsv file of cleaned tweets
index = 0
with open('task1_training_cleaned.tsv', mode='w') as tsvfile:
    tsvwriter = csv.writer(tsvfile, delimiter='\t')

    index = 0
    for tweet in clean_tweets:
        tsvwriter.writerow([id[index], target[index], tweet])

        index += 1
tsvfile.close()

Reading twitter - 1grams ...
Reading twitter - 2grams ...


/miniconda3/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter - 1grams ...
Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [16]:
import pandas as pd

tweetdata = pd.read_csv('task1_training_cleaned.tsv', sep= '\t', names=['id','label','tweet'])
adrdata=pd.read_csv("/Users/ooduyebo/Desktop/REAC18Q4.txt", sep= "$", names=['pid',"caseid", 'reaction','reaction'])
newadrdata= pd.read_csv("/Users/ooduyebo/Desktop/meddra_all_se.txt", names= ["reaction"])

#tweetdata
#print(newadrdata)

#Adverse Drug reactions- 1.2m
adrdata=adrdata.drop([0])
adrdata=adrdata.drop(['reaction.1','pid', 'caseid'], axis=1)
adrdata=adrdata['reaction'].str.lower()

#changing a few things
newadrdata= newadrdata["reaction"].str.lower()

#converting it back to a dataframe
newadrdata= newadrdata.to_frame()




#print(type(adrdata))
#adrdata=adrdata.to_frame()
#adrdata






re.compile(r'\b(?:reaction)\b', re.UNICODE)

In [78]:
drugdata= pd.read_csv("drug_names.tsv", sep= '\t', names=['id','drug'])

In [79]:
drugdata

,id,drug
0,CID100000085,carnitine
1,CID100000119,gamma-aminobutyric
2,CID100000137,5-aminolevulinic
3,CID100000143,leucovorin
4,CID100000146,5-methyltetrahydrofolate
5,CID100000158,PGE2
6,CID100000159,prostacyclin
7,CID100000160,prostaglandin
8,CID100000175,acetate
9,CID100000187,acetylcholine


In [80]:
drugdata=drugdata.drop(['id'], axis=1)
drugdata=drugdata['drug'].str.lower()
drugdata= drugdata.to_frame()
drugdata


,drug
0,carnitine
1,gamma-aminobutyric
2,5-aminolevulinic
3,leucovorin
4,5-methyltetrahydrofolate
5,pge2
6,prostacyclin
7,prostaglandin
8,acetate
9,acetylcholine


In [61]:

tweetdata['side_effect'] = tweetdata.apply(lambda row: '1' if 'side effect'  in row['tweet'] 
                                           or  'side-effect'in row['tweet'] else '0', 
                                           axis=1)
tweetdata.head()





,id,label,tweet,reaction,side_effect
0,does not equalB339867818843594756,0,<user> doctor christian </user> scared to star...,[],0
1,349294537367236611,0,"<user> intuitive gal 1 </user> ok , if you sto...",[],0
2,354256195432882177,0,novartis announces secukinumab ( ain 457 ) dem...,[psoriasis],0
3,352456944537178112,1,""" u wailed all night ; now y'r disembodied sob...",[],0
4,332479707004170241,0,<user> ira paps </user> you are so fucking sel...,[],0


In [60]:
print(tweetdata["tweet"][14])

finding out i am allergic to fluoxetine was a bit of a shock tho . mind , only someone from yorkshire could be actually allergic to prozac


In [35]:
#putting all my tweets in a list
tweet_list= tweetdata["tweet"].tolist()
#tweet_list

reaction_list= newadrdata["reaction"].tolist()

#remove duplicates
reaction_list= list(set(reaction_list))








In [83]:
drug_list= drugdata["drug"].tolist()
drug_list= list(set(drug_list))
drug_list

['abacavir-lamivudine',
 'dothiepin',
 'gemfibrozil',
 'cefotaxime',
 'bexarotene',
 'darifenacin',
 'tetracycline',
 'etidronate',
 'colimycin',
 'penbutolol',
 'perindopril',
 'florbetaben',
 'pge1',
 'miconazole',
 'lodoxamide',
 'regorafenib',
 'varenicline',
 'suvorexant',
 'eltrombopag',
 'timentin',
 'amsacrine',
 'monobenzone',
 'gamma-aminobutyric',
 'danazol',
 'bretylium',
 'doxepin',
 'echothiophate',
 'choline',
 'o291',
 'nortriptyline',
 'ly146032',
 'hetastarch',
 'travoprost',
 'nepafenac',
 'progesterone',
 '4-ap',
 'chloroquine',
 'benzathine',
 'carisoprodol',
 'capreomycin',
 'capecitabine',
 'elvitegravir',
 'ethosuximide',
 'tolbutamide',
 'codeine',
 'cimetidine',
 'brimonidine',
 'fenoldopam',
 'abiraterone',
 'nabumetone',
 'tenoxicam',
 'isomannide',
 'trisequens',
 'azelaic',
 'acyclovir',
 'ciclopirox',
 'prednisone',
 'itraconazole',
 'terlipressin',
 'brinzolamide',
 'cerivastatin',
 'rifampicin',
 'vancocine',
 'd-penicillamine',
 'methysergide',
 'epina

In [54]:
import string


translator= str.maketrans('','',string.punctuation)

cleaned_reaction= []


for sub in reaction_list:
    cleaned = sub.translate(translator)
    cleaned_reaction.append(cleaned)

cleaned_reaction

    


         

['sprue',
 'scleral discolouration',
 'highpitched crying',
 'vestibular neuronitis',
 'concussion',
 'esophageal erosions',
 'multiple allergies',
 'telangiectasis facial',
 'corneal disorder',
 'vascular headache',
 'pustule',
 'movements involuntary',
 'electrocardiogram change',
 'cloudy urine',
 'hypoaesthesia',
 'burning mouth syndrome',
 'hepatitis symptom',
 'jaw stiffness',
 'pyogenic granuloma',
 'multiple congenital abnormalities',
 'renal tubular disorder',
 'chylothorax',
 'decreased interest',
 'low density lipoprotein increased',
 'causalgia',
 'mantle cell lymphoma',
 'ovarian hyperstimulation syndrome',
 'congenital joint malformation',
 'large intestinal haemorrhage',
 'oedema due to cardiac disease',
 'malignant neoplasm progression',
 'fluoride increased',
 'arthritis aggravated',
 'obstipation',
 'carotid artery occlusion',
 'accidental exposure to product',
 'red blood cell count increased',
 'steroid psychosis',
 'candida infection',
 'antineutrophil cytoplasmic 

In [85]:

translator= str.maketrans('-','-',string.punctuation)

cleaned_drug= []


for sub in drug_list:
    cleaned = sub.translate(translator)
    cleaned_drug.append(cleaned)

cleaned_drug


['abacavirlamivudine',
 'dothiepin',
 'gemfibrozil',
 'cefotaxime',
 'bexarotene',
 'darifenacin',
 'tetracycline',
 'etidronate',
 'colimycin',
 'penbutolol',
 'perindopril',
 'florbetaben',
 'pge1',
 'miconazole',
 'lodoxamide',
 'regorafenib',
 'varenicline',
 'suvorexant',
 'eltrombopag',
 'timentin',
 'amsacrine',
 'monobenzone',
 'gammaaminobutyric',
 'danazol',
 'bretylium',
 'doxepin',
 'echothiophate',
 'choline',
 'o291',
 'nortriptyline',
 'ly146032',
 'hetastarch',
 'travoprost',
 'nepafenac',
 'progesterone',
 '4ap',
 'chloroquine',
 'benzathine',
 'carisoprodol',
 'capreomycin',
 'capecitabine',
 'elvitegravir',
 'ethosuximide',
 'tolbutamide',
 'codeine',
 'cimetidine',
 'brimonidine',
 'fenoldopam',
 'abiraterone',
 'nabumetone',
 'tenoxicam',
 'isomannide',
 'trisequens',
 'azelaic',
 'acyclovir',
 'ciclopirox',
 'prednisone',
 'itraconazole',
 'terlipressin',
 'brinzolamide',
 'cerivastatin',
 'rifampicin',
 'vancocine',
 'dpenicillamine',
 'methysergide',
 'epinastin

In [55]:
r=re.compile(r'\b(?:%s)\b' % '|'.join(cleaned_reaction))

In [86]:
newr=re.compile(r'\b(?:%s)\b' % '|'.join(cleaned_drug))

In [87]:
newr

re.compile(r'\b(?:abacavirlamivudine|dothiepin|gemfibrozil|cefotaxime|bexarotene|darifenacin|tetracycline|etidronate|colimycin|penbutolol|perindopril|florbetaben|pge1|miconazole|lodoxamide|regorafenib|varenicline|suvorexant|eltrombopag|timentin|amsacrine|monobenzone|gammaaminobutyric|danazol|bretylium|doxepin|echothiophate|choline|o291|nortriptyline|ly146032|hetastarch|travoprost|nepafenac|progesterone|4ap|chloroquine|benzathine|carisoprodol|capreomycin|capecitabine|elvitegravir|ethosuximide|tolbutamide|codeine|cimetidine|brimonidine|fenoldopam|abiraterone|nabumetone|tenoxicam|isomannide|trisequens|azelaic|acyclovir|ciclopirox|prednisone|itraconazole|terlipressin|brinzolamide|cerivastatin|rifampicin|vancocine|dpenicillamine|methysergide|epinastine|stavudine|loratadine|losartan|famotidine|sorafenib|apixaban|raltegravir|fluoxymesterone|nicardipine|cyclizine|idarubicin|linezolid|bimatoprost|alosetron|fentanyl|ramelteon|ertapenem|amd3100|som230|imiquimod|ethambutol|desipramine|thalidomide|

In [56]:
r

re.compile(r'\b(?:sprue|scleral discolouration|highpitched crying|vestibular neuronitis|concussion|esophageal erosions|multiple allergies|telangiectasis facial|corneal disorder|vascular headache|pustule|movements involuntary|electrocardiogram change|cloudy urine|hypoaesthesia|burning mouth syndrome|hepatitis symptom|jaw stiffness|pyogenic granuloma|multiple congenital abnormalities|renal tubular disorder|chylothorax|decreased interest|low density lipoprotein increased|causalgia|mantle cell lymphoma|ovarian hyperstimulation syndrome|congenital joint malformation|large intestinal haemorrhage|oedema due to cardiac disease|malignant neoplasm progression|fluoride increased|arthritis aggravated|obstipation|carotid artery occlusion|accidental exposure to product|red blood cell count increased|steroid psychosis|candida infection|antineutrophil cytoplasmic antibody positive|capillary fragility|manic psychosis|developmental coordination disorder|progesterone|atelectasis|dysfunction thyroid|feeli

In [58]:
tweetdata['reaction']=tweetdata['tweet'].str.findall(r)


In [62]:
tweetdata.head()

,id,label,tweet,reaction,side_effect
0,does not equalB339867818843594756,0,<user> doctor christian </user> scared to star...,[],0
1,349294537367236611,0,"<user> intuitive gal 1 </user> ok , if you sto...",[],0
2,354256195432882177,0,novartis announces secukinumab ( ain 457 ) dem...,[psoriasis],0
3,352456944537178112,1,""" u wailed all night ; now y'r disembodied sob...",[],0
4,332479707004170241,0,<user> ira paps </user> you are so fucking sel...,[],0


In [88]:
tweetdata['drug']=tweetdata['tweet'].str.findall(newr)



In [89]:
tweetdata

,id,label,tweet,reaction,side_effect,drug
0,does not equalB339867818843594756,0,<user> doctor christian </user> scared to star...,[],0,[fluoxetine]
1,349294537367236611,0,"<user> intuitive gal 1 </user> ok , if you sto...",[],0,[]
2,354256195432882177,0,novartis announces secukinumab ( ain 457 ) dem...,[psoriasis],0,[]
3,352456944537178112,1,""" u wailed all night ; now y'r disembodied sob...",[],0,[]
4,332479707004170241,0,<user> ira paps </user> you are so fucking sel...,[],0,[lamotrigine]
5,340660708364677120,1,"not that anyone noticed , but my <hashtag> amb...",[],0,[]
6,341588695440310272,0,oh . i thought the cymbalta commercial was a j...,[],0,[]
7,344189055308816384,0,<user> c 4 dispatches </user> ek <elongated> ....,[],0,[]
8,345005092166041600,0,at the cypriot art and archaeology exhibition ...,[],0,[]
9,331692494251233280,0,some fantastic talks at this years <hashtag> p...,[],0,[fluoxetine]
